## Assinatura digital de RSA

### Criação de uma Chave Pública e uma Chave Privada

Dado um número de bits, são calculados, aleatoriamente, dois primos, p e q diferentes e, que divididos por 4 dêem resto 3.

$$p \equiv 3 (mod \:4)$$
$$q \equiv 3 (mod \:4)$$


##### Chave pública

Posteriormente, é calculado uma $\textbf{chave pública}$ que vai ser um par onde o primeiro elemento será:
$$ n = p * q$$

e o segundo elemento será um valor $\textit{e}$ obtido aleatoriamente no intervalo $[2, \phi - 1]$ onde $\phi = (p - 1) * (q - 1)$, sendo que, o $\textit{e}$ e o $\phi$ são primos relativos.

$$\textbf{Chave pública = (n,e)}$$


##### Chave privada

A chave privada também vai ser retornado como um par no qual o primeiro elemento é:
$$ d = e^{-1} \mod (\phi)$$

e o segundo elemento será o valor $\textit{n}$

$$\textbf{Chave privada = (d,n)}$$

In [2]:
def RSA(nbits):
    p = random_prime(2^(nbits//2),2^(nbits//2-1))
    while mod(p,4)!=3:
        p = random_prime(2^(nbits//2),2^(nbits//2-1))
    
    q = random_prime(2^(nbits//2),2^(nbits//2-1))
    while mod(q,4)!=3 and p != q:
        q = random_prime(2^(nbits//2),2^(nbits//2-1))
    
    n = p * q
    
    phi = (p-1)*(q-1)
    
    e = randint(2, phi-1)
    while gcd(e, phi)!=1:
        e = randint(2, phi-1)
    
    d = power_mod(e, -1, phi)
    
    public_key = (n, e)
    private_key = (d,n)
        
    return public_key, private_key

## Encriptação

Na função $\textbf{RSA_encrypt}$ é passado como argumento uma $\textit{signMessage}$ que será um par $(s, mensagem)$, onde $\textit{s}$ será: $$s = h^d (\mod n)$$ no qual $\textit{h}$ é o $\textbf{hashcode}$ da mensagem inicial $( h = H(mensagem))$.

Para além deste argumento, a $\textbf{pubKey}$ também é dada como argumento.

Esta função irá retornar: $$s^e (\mod n)$$

In [3]:
def RSA_Encrypt(signMessage, pubKey):
    n,e = pubKey
    s,_ = signMessage
    return power_mod(s,e,n)

### Prova

Para provar a correção do RSA usamos o teorema de Euler, $\textbf{eulerProof}$, onde queremos mostrar que:
$$ m == m^{e*d} (\mod n)$$

In [12]:
def eulerProof(m, pubKey, privKey):
    return m == power_mod(m, pubKey[1]*privKey[0],pubKey[0])

In [19]:
import hashlib

@interact
def digitalSignatureRSA(bits=slider(260, 1028, 4, 8, 'Numero de bits', False), inputMessage=input_box('"Nobody expects the Spanish Inquisition"', label="$message$",width=20)):
    
    print("Bits: ", bits)
    
    HH =hashlib.sha256(str(inputMessage).encode()).hexdigest()
    h = ZZ('0x'+HH)
    
    pubKey, privKey = RSA(bits)
    
    n,e = pubKey
    d,n = privKey
    
    s = power_mod(h, d, n)

    signMessage = (s, "AaAaAa")
    
    h1 = RSA_Encrypt(signMessage, pubKey)
    
    
    print("A assinatura h1 == h(m) é válida?\n",h == h1)

    proof = eulerProof(h, pubKey, privKey)
    
    print("Satisfaz o teorema de Euler?\n", proof)

Interactive function <function digitalSignatureRSA at 0x7f283f7570d0> with 2 widgets
  bits: TransformIntSlide…

## Verificação da hash criptográfica

Para verificar a hash criptográfica tem de satisfazer 3 propriedades.
<ol type="a">
    <li><b>Preimage Resistant</b> - Deve ser difícil encontrar a mensagem com o valor do hash dado.</li>
    <li><b>Collision Resistant</b> - Deve ser difícil encontrar duas mensagens com o mesmo valor de hash.</li> 
    <li><b>Second Preimage Resistant</b> - Dado uma mensagem, deve ser difícil encontrar outra
mensagem com o mesmo valor hash.</li>
</ol>

### Requirement for preimage resistance

A propriedade one-way é a única maneira de parar um criptoanalista de preparar uma mensagem com uma determinada assinatura. Numa função hash que não tem a propriedade one-way, deparamo-nos com a seguinte adversidade:
        <li>A Eve calcula:
            <b>h1 = $r^e$ (mod N)</b>
        <li>Eve também calcula a pré-imagem de h1 sob h (lembrar de que estamos a assumir que h não têm a propriedade one-way),ou seja,Eve calcula:
            <b> m = $h^{-1}$(h1)</b>

Eve tem agora a sua assinatura $(m,r)$ na mensagem $m$. Essa tal falsificação é chamada de falsificação existencial no sentido em que o invasor pode não ter qualquer controlo sobre o conteúdo da mensagem na qual ele obteve uma assinatura digital.

### Requirement for collision resistance

Esta propriedade é necessária para evitar o seguinte ataque, que é feito pelo <i>signer</i> legítimo.
<ul>
    <li>O <i>signer</i> escolhe duas mensagens <b><i>m</i></b> e <b><i>m'</i></b> com h(<b><i>m</i></b>) = h(<b><i>m'</i></b>)</li>
    <li>Eles assinam <b><i>m</i></b> e geram o resultado da assinatura (<b><i>m</i></b>, <b><i>s</i></b>)</li>
    <li>Depois eles recusam a assinatura, dizendo que era a assinatura da mensagem <b><i>m'</i></b></li>
</ul>
Exemplo:

Um <i>signer</i> pode escolher entre um <b><i>m</i></b>, sendo este um cheque eletrónico de 1000€, e um <b><i>m'</i></b> que é o cheque eletrónico de 10€. São duas mensagens diferentes com a mesma hash e, independentemente da escolha que fizer é indiferente se faz a assinatura ou não, pois tanto  <b><i>m</i></b> e  <b><i>m'</i></b> têm a mesma hash.

### Requirement for second preimage resistance

Esta propriedade é necessária para evitar o seguinte ataque:
<ul>
    <li>Um infrator obtém a assinatura <i>(m,s)</i> de uma mensagem <i>m</i>.</li>
    <li>O infrator encontra outra mensagem <i> m' </i> através de <i>h(m') = h(m)</i>.</li>
    <li>O infrator tem agora a assinatura <i>(m',s)</i> através da mensagem <i>m'</i>.</li>
</ul>

Um infrator consegue obter uma assinatura <i>(m,s)</i> através de uma mensagem <i>m</i> executando o seguinte cálculo:
$$s = m^d \mod n$$
onde d é obtido a partir de uma <i>PrivKey</i>.

Após isso o infrator calcula o <i>hashcode</i> da mensagem inicial e tenta encontrar uma nova mensagem com o mesmo <i>hashcode</i> no entanto, como o algoritmo utilizado para encriptar a mensagem é o <b>SHA256</b>, apesar de ser teoricamente possível encontrar duas mensagens com o mesmo <i>hascode</i>, é muito improvável isso acontecer.

## Chaum's Blind Signature Scheme

O esquema de assinatura cega de Chaum é um esquema baseado em RSA, adaptado para assinaturas cegas. No protocolo a seguir, assumimos que o Bob configurou uma chave RSA pública (e, n) com a chave privada correspondente (d, n), de modo a que as funções de assinatura RSA de Bob sejam $S_{B}(m) = m^d$.
<ul>  
    <li><i>Initial setup:</i> <b>Alice</b> obtém a chave pública do <b>Bob</b> <i>(e, n)</i> e escolhe uma chave pública aleatória <i>k</i>, tal que $0 < k < n$ e GCD$\textit{(e,n)} = 1$.   </li>
    <li><i>Blinding:</i> <b>Alice</b> calcula $m'= m * k^e (\mod n)$, e envia $m'$ para o <b>Bob</b>.</li>
      <li><i>Signing: </i><b>Bob</b> calcula $s' = (m')^d (\mod n)$, que ele envia de volta para a <b>Alice.</b></li>
      <li><i>Unblinding:</i> <b>Alice</b> calcula $s = k^{-1} * s'$, que é igual a $s = S_{B}(m) = m^d$.</li>
</ul>

Esta assinatura é verdadeira se:

$$s = m^d \models true$$

In [14]:
import random
import hashlib

def initialSetup(n):
    while(True):
        k = random.randint(0, n)
        if (gcd(k,n) == 1):
            break
    return k

In [15]:
def blinding(m,k,pubKey):
    n,e = pubKey
    Zn = IntegerModRing(n)
    a = Zn(k)
    return Zn(m * a^e)

In [16]:
def signing(m1,privKey):
    d,n = privKey
    Zn = IntegerModRing(n)
    return Zn(m1^d)

In [17]:
def unblinding(k, s1,n):
    Zn = IntegerModRing(n)
    a = Zn(k)
    s = pow (a, -1)
    return Zn(s * s1)

In [18]:
@interact
def digitalSignatureRSA(bits=slider(256, 1028, 4, 8, 'Numero de bits', False), inputMessage=input_box('"Nobody expects the Spanish Inquisition"', label="$message$",width=20)):
    print("Bits: ", bits)
    
    HH =hashlib.sha256(str(inputMessage).encode()).hexdigest()
    h = ZZ('0x'+HH)
    print(inputMessage)
    
    pubKey, privKey = RSA(bits)
    
    k = initialSetup(pubKey[0])
    
       
    m1 = blinding(h,k,pubKey)
    
    
    s1 = signing(m1,privKey)
    
    
    result = unblinding(k, s1, pubKey[0])
    
    signingS = power_mod(h, privKey[0], pubKey[0])
    

    print("s == m^d?\n",signingS==result)

Interactive function <function digitalSignatureRSA at 0x7f283f790af0> with 2 widgets
  bits: TransformIntSlide…